### Import Necessary Dependencies

In [ ]:
import os
import time as time

import numpy as np # Optimizing matrix operations
import random
np.random.seed(105111) # to generate same random number all the time
import matplotlib.pyplot as plt # Data Visualization
import matplotlib.gridspec as gridspec
import math # Math module to deal with math operations 
%matplotlib inline # To Present the images on the screen itself

from PIL import Image # Used to open image content into an array
import time
from datetime import timedelta
import pandas as pd # data Manipulation
from sklearn.metrics import accuracy_score # Finding accuracy of the trained model
from matplotlib import pyplot
import skimage.morphology as morp
from skimage.filters import rank

import keras

from keras.models import Sequential
from keras.layers import Dense, Flatten # Layers included in the model architecture
from keras.layers import Dropout # Regularize our model
from keras.layers.normalization import BatchNormalization # Normalize the data
from keras import regularizers # new! 
from keras.optimizers import SGD # Optimizer
from keras.layers import Flatten, Conv2D, MaxPooling2D # Different layers in CNN
from keras.callbacks import ModelCheckpoint # For saving the best weights wrt validation erro

import cv2

### Unzip the Dataset Folder

In [ ]:
!wget https://d17h27t6h515a5.cloudfront.net/topher/2017/February/5898cd6f_traffic-signs-data/traffic-signs-data.zip
!unzip -q traffic-signs-data.zip

### Read the Dataset in separate subsets
Download the dataset from [here](https://d17h27t6h515a5.cloudfront.net/topher/2017/February/5898cd6f_traffic-signs-data/traffic-signs-data.zip). This is a pickled dataset in which we've already resized the images to 32x32.
We already have three .p files of 32x32 resized images:
train.p: The training set.
test.p: The testing set.
valid.p: The validation set.
We will use Python pickle to load the data.

In [ ]:
training_file = "/content/train.p"
testing_file = "/content/test.p"
validation_file = "/content/valid.p"

### Load the Separate Subsets of the Initial Dataset
The pickled data is a dictionary with 4 key/value pairs:
1. 'features' is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
2. 'labels' is a 1D array containing the label/class id of the traffic sign. The file signnames.csv contains id -> name mappings for each id.
3. 'sizes' is a list containing tuples, (width, height) representing the original width and height the image.
4. 'coords' is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image.

The code snippets below will provide a basic summary of the Dataset.

In [ ]:
import pickle

with open(training_file, mode='rb') as f:  # Training Dataset
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:  # Testing Dataset
    test = pickle.load(f)
with open(validation_file, mode='rb') as f:  # Validation Dataset
    valid = pickle.load(f)
    
X_train, y_train = train['features'], train['labels'] # Separate features and labels for training dataset
X_test, y_test = test['features'], test['labels'] # Separate features and labels for testing dataset
X_valid , y_valid = valid['features'], valid['labels'] # Separate features and labels for Validation dataset

### Understand the data

In [ ]:
print("Training Set:", len(X_train)) # Length of training dataset
print("Test Set:", len(y_test)) # Length of testing dataset
print("Validation Set:", len(X_valid))  # Length of validation dataset
print("Image Dimensions:", np.shape(X_train[1]))  # Dimension of the images in the dataset
print("Number of classes:", len(np.unique(y_train))) # Number of classes/possible outputs for images
n_classes = len(np.unique(y_train)) # Store the number of classes in n_classes variable

### Use matplotlib to plot sample images from each subset

In [ ]:
# And finally, we will use numpy to plot a histogram of the count of images in each unique class.
def histogram_plot(dataset, label):
    """
    Plots a histogram of the input data.
        Parameters:
            dataset: Input data to be plotted as a histogram.
            lanel: A string to be used as a label for the histogram.
    """
    hist, bins = np.histogram(dataset, bins=n_classes)
    width = 0.7 * (bins[1] - bins[0])
    center = (bins[:-1] + bins[1:]) / 2
    plt.bar(center, hist, align='center', width=width)
    plt.xlabel(label)
    plt.ylabel("Image count")
    plt.show()

In [ ]:
# Plotting histograms of the count of each sign
histogram_plot(y_train, "Training examples")
histogram_plot(y_test, "Testing examples")
histogram_plot(y_valid, "Validation examples")

### Basic Descriptive analysis

In [ ]:
# Checking for Class Bias 

unique_elements, counts_elements = np.unique(y_train, return_counts = True)
print(np.asarray((unique_elements, counts_elements)))

pyplot.bar( np.arange( 43 ), counts_elements, align='center',color='red' )
pyplot.xlabel('Class Numbers for different classes')
pyplot.ylabel('Number of Training data for that specific class')
pyplot.xlim([-1, 43])

pyplot.show()

print(" ")
print("We can definitely see class bias issue as certain classes are under represented")

# Print some images from the dataset
print(" ")
print(" ")
print("Traffic Sign Images")
fig, axs = plt.subplots(8,5, figsize=(10, 10))
fig.subplots_adjust(hspace = .5, wspace=.001)
axs = axs.ravel()
for i in range(40):
    index = random.randint(0, len(X_train))
    image = X_train[index]
    axs[i].axis('off')
    axs[i].imshow(image)
    axs[i].set_title(y_train[index])

### **Model Testing without any preprocessing -**


### Establishing Baseline Neural Network Architecture

In [ ]:
from keras.layers.normalization import BatchNormalization 

model = Sequential() # Gives a blank box as the output(Load the model) and we use further this to add the fully connected layers in it
model.add(Dense(128, activation='relu', input_shape=(32*32*3,))) # Input layer
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5)) # 50% neurons are dropped
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5)) 
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(n_classes, activation='softmax')) # Apply softmax activation at the Final layer to find probability distribution of differ classes

### Analyzing model Summary and Compile it

In [ ]:
model.summary() 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

### Make train and validation dataset for baseline model

In [ ]:
X_train_baseline = X_train.reshape(len(X_train), 32*32*3).astype('float32')
X_valid_baseline = X_valid.reshape(len(X_valid), 32*32*3).astype('float32')
# From the keras.utils package, we use to_categorical method to convert the labels present in y_train and y_test into one-hot encoding.
from tensorflow.keras.utils import to_categorical 
y_train_baseline = to_categorical(y_train, n_classes)
y_valid_baseline = to_categorical(y_valid, n_classes)

### Training or fitting the Model

In [ ]:
model.fit(X_train_baseline, y_train_baseline, batch_size=128, epochs=10
, verbose=1, validation_data=(X_valid_baseline, y_valid_baseline))

### Make Testing dataset for baseline model

In [ ]:
X_test_baseline = X_test.reshape(len(X_test), 32*32*3).astype('float32') # reshaping the images according to our images in training dataset
y_test_baseline = to_categorical(y_test, n_classes) # actual output for all the images to evaluate the model

### Evaluate the model

In [ ]:
Prediction_test = model.evaluate(X_test_baseline, y_test_baseline, verbose=0)
print("Results for Dense fully connected network (baseline model) on the test data \n")
print("%s- %.3f" % (model.metrics_names[0], Prediction_test[0]))
print("%s- %.3f" % (model.metrics_names[1], Prediction_test[1]))

### Data Augmentation

In [ ]:
# Function to augment the images (make more images other than given images in the dataset)
def data_augment(image):
    rows= image.shape[0]
    cols = image.shape[1]
    
    # New Image formed with the help of rotation
    M_rot = cv2.getRotationMatrix2D((cols/2,rows/2),10,1)
    
    # New Image formed with the help of Translation
    M_trans = np.float32([[1,0,3],[0,1,6]])
    
    
    img = cv2.warpAffine(image,M_rot,(cols,rows)) # rotation
    img = cv2.warpAffine(img,M_trans,(cols,rows)) # translation
    
    # New Image formed with the help of Bilateral filtering
    img = cv2.bilateralFilter(img,9,75,75)
    return img

### Increase training set by Data augmentation and making all class size similar


In [ ]:
classes = 43

X_train_final = X_train
y_train_final = y_train
X_aug_1 = []
Y_aug_1 = []

for i in range(0,classes):
    
    class_records = np.where(y_train==i)[0].size
    max_records = 4000
    if class_records != max_records:
        ovr_sample = max_records - class_records
        samples = X_train[np.where(y_train==i)[0]]
        X_aug = []
        Y_aug = [i] * ovr_sample
        
        for x in range(ovr_sample):
            img = samples[x % class_records]
            trans_img = data_augment(img)
            X_aug.append(trans_img)
            
        X_train_final = np.concatenate((X_train_final, X_aug), axis=0)
        y_train_final = np.concatenate((y_train_final, Y_aug)) 
        
        Y_aug_1 = Y_aug_1 + Y_aug
        X_aug_1 = X_aug_1 + X_aug

### Check class bias after augmentation

In [ ]:
unique_elements, counts_elements = np.unique(y_train_final, return_counts = True)
print(np.asarray((unique_elements, counts_elements)))

pyplot.bar( np.arange( 43 ), counts_elements, align='center',color='green' )
pyplot.xlabel('Class')
pyplot.ylabel('No of Training data')
pyplot.xlim([-1, 43])

pyplot.show()

### Shape of the datasets after data augmentation

In [ ]:
print(len(X_train)) # features of Training dataset length before augmentation
print(len(X_train_final)) # features of Training dataset length after augmentation
print(len(y_train)) # labels of Training dataset length before augmentation
print(len(y_train_final)) # labels of Training dataset length after augmentation

### View images after data augmentation

In [ ]:
import matplotlib.pyplot as plt
import random

%matplotlib inline
print(" ")
print(" ")
print("Present some of the images to understand our applied transformations")
fig, axs = plt.subplots(8,5, figsize=(10, 10))
fig.subplots_adjust(hspace = .5, wspace=.001)
axs = axs.ravel()
for i in range(40):
    index = random.randint(0, len(X_aug_1))
    image = X_aug_1[index]
    axs[i].axis('off')
    axs[i].imshow(image)
    axs[i].set_title(Y_aug_1[index])

### Model Testing after Augmentation

In [ ]:
X_train_aug = X_train_final.reshape(len(X_train_final), 32*32*3).astype('float32')
X_valid_aug = X_valid.reshape(len(X_valid), 32*32*3).astype('float32')
y_train_aug = to_categorical(y_train_final, n_classes)
y_valid_aug = to_categorical(y_valid, n_classes)

In [ ]:
import h5py
filepath="German_Traffic_DenseNetworkModel_AfterAugmentation.h5"
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
model.fit(X_train_aug, y_train_aug, batch_size=128, epochs=50, verbose=1,callbacks=callbacks_list,validation_data=(X_valid_aug, y_valid_aug))

In [ ]:
X_test_aug = X_test.reshape(len(X_test), 32*32*3).astype('float32')
y_test_aug = to_categorical(y_test, n_classes)

In [ ]:
# model.load_weights("German_Traffic_DenseNetworkModel_AfterAugmentation.h5")
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
Pred = model.evaluate(X_test_aug, y_test_aug, verbose=0)
print("Dense fully connected network results on the test data - After Data Augmentation ")
print(" ")
print("%s- %.2f" % (model.metrics_names[0], Pred[0]))
print("%s- %.2f" % (model.metrics_names[1], Pred[1]))

### Data preprocessing 
In this step, we will apply several preprocessing steps to the input images to achieve the best possible results.
We will use the following preprocessing techniques:
Shuffling,
Grayscaling,
Local Histogram Equalization,
Normalization.

### Shuffling of Images

In [ ]:
from sklearn.utils import shuffle
X_train_aug, y_train_aug = shuffle(X_train_aug, y_train_aug)

### Convert the Images into Grey Scale instead of RGB

In [ ]:
def gray_scale(image):
    
    return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

gray_images_data = list(map(gray_scale, X_train_final))


%matplotlib inline
print(" ")
print(" ")
print("Present few images to familiarize with the dataset")
fig, axs = plt.subplots(8,5, figsize=(10, 10))
fig.subplots_adjust(hspace = .5, wspace=.001)
axs = axs.ravel()
for i in range(40):
    index = np.random.randint(0, len(gray_images_data))
    image = gray_images_data[index]
    axs[i].axis('off')
    axs[i].imshow(image,cmap='gray')
    axs[i].set_title(y_train_final[index])

### Local Histogram Localization

In [ ]:
import skimage.morphology as morp
from skimage.filters import rank

def local_histo_equalize(image):
    
    kernel = morp.disk(30)
    img_local = rank.equalize(image, selem=kernel)
    return img_local


local_histo = np.array(list(map(local_histo_equalize, gray_images_data)))
%matplotlib inline
print(" ")
print(" ")
print("Let's view few images after applying local histogram equilization and Gray scaling")
fig, axs = plt.subplots(8,5, figsize=(10, 10))
fig.subplots_adjust(hspace = .5, wspace=.001)
axs = axs.ravel()
for i in range(40):
    index = np.random.randint(0, len(local_histo))
    image = local_histo[index]
    axs[i].axis('off')
    axs[i].imshow(image,cmap = 'gray')
    axs[i].set_title(y_train_final[index])

### Function to preprocess the data according to above data processing

In [ ]:
def preprocess(data):
    gray_images = list(map(gray_scale, data))
    equalized_images = list(map(local_histo_equalize, gray_images))
    n_training = data.shape
    normalized_images = np.zeros((n_training[0], n_training[1], n_training[2]))
    for i, img in enumerate(equalized_images): # Normalization step 
        normalized_images[i] = np.divide(img,255)
    normalized_images = normalized_images[..., None]
    return normalized_images

### Apply Data preprocessing steps to the training and the validation data

In [ ]:
X_train_preprocessed = preprocess(X_train_final)
X_valid_preprocessed = preprocess(X_valid)

In [ ]:
X_train_preprocessed_dn = X_train_preprocessed.reshape(len(X_train_preprocessed), 32*32*1).astype('float32')
X_valid_preprocessed_dn = X_valid_preprocessed.reshape(len(X_valid_preprocessed), 32*32*1).astype('float32')

In [ ]:
y_train_final_dn = to_categorical(y_train_final, n_classes)
y_valid_final_dn = to_categorical(y_valid, n_classes)

### Check the shape of the datasets after all preprocessing

In [ ]:
print(X_train_preprocessed_dn.shape)
print(X_valid_preprocessed_dn.shape)
print(y_train_final_dn.shape)
print(y_valid_final_dn.shape)

### Compile and fit the model after preprocessing

### Neural network architecture

In [ ]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(32*32*1,)))
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(n_classes, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) # Model Compilation

### Save model with Checkpoint

In [ ]:
filepath="German_Traffic_DenseNetworkModel_final.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
model.fit(X_train_preprocessed_dn, y_train_final_dn, batch_size=128, epochs=50, verbose=1,callbacks=callbacks_list,validation_data=(X_valid_preprocessed_dn, y_valid_final_dn))

### Prepare test data for final results

In [ ]:
# Prepare the Test data with all the preprocessing

X_test_preprocessed = preprocess(X_test)
X_test_preprocessed_dn = X_test_preprocessed.reshape(len(X_test_preprocessed), 32*32*1).astype('float32')
y_test_final_dn = to_categorical(y_test, n_classes)

### Load the best model from the validation data results


In [ ]:

model.load_weights("German_Traffic_DenseNetworkModel_final.hdf5")

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
Pred = model.evaluate(X_test_preprocessed_dn, y_test_final_dn, verbose=0)
print("Dense fully connected network results on the test data")
print(" ")
print("%s- %.2f%%" % (model.metrics_names[0], Pred[0]*100))
print("%s- %.2f%%" % (model.metrics_names[1], Pred[1]*100))

### Convolutional Networks

In [ ]:
# Prepare data for Conv nets
X_train_preprocessed_cn = X_train_preprocessed.reshape(len(X_train_preprocessed), 32,32,1).astype('float32')
X_valid_preprocessed_cn = X_valid_preprocessed.reshape(len(X_valid_preprocessed), 32,32,1).astype('float32')
X_test_preprocessed_cn = X_test_preprocessed.reshape(len(X_test_preprocessed), 32,32,1).astype('float32')

In [ ]:
y_train_final_cn = to_categorical(y_train_final, n_classes)
y_valid_final_cn = to_categorical(y_valid, n_classes)
y_test_final_cn = to_categorical(y_test, n_classes)

In [ ]:
model_conv = Sequential()
from keras.layers import Flatten, Conv2D, MaxPooling2D
## If You preprocessed with gray scaling and local histogram equivalization then input_shape = (32,32,1) else (32,32,3)
model_conv.add(Conv2D(32, kernel_size=(3, 3),activation='relu', input_shape=(32, 32, 1)))
model_conv.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model_conv.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model_conv.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model_conv.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model_conv.add(Dropout(0.25))
model_conv.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model_conv.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model_conv.add(Dropout(0.5))
model_conv.add(Flatten())
model_conv.add(Dense(128, activation='relu'))
model_conv.add(Dropout(0.5))
model_conv.add(Dense(n_classes, activation='softmax'))

In [ ]:
model_conv.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
filepath="German_Traffic_ConvNetworkModel.h5"
checkpoint_conv = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list_conv = [checkpoint_conv]

In [ ]:
model_conv.fit(X_train_preprocessed_cn, y_train_final_cn, batch_size=128, epochs=3, verbose=1,callbacks=callbacks_list_conv,validation_data=(X_valid_preprocessed_cn, y_valid_final_cn))

In [ ]:
# model_conv.load_weights("German_Traffic_ConvNetworkModel.h5")
model_conv.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
Pred_conv = model_conv.evaluate(X_test_preprocessed_cn, y_test_final_cn, verbose=0)
print("Dense fully connected network results on the test data")
print(" ")
print("%s- %.2f" % (model.metrics_names[0], Pred_conv[0]))
print("%s- %.2f" % (model.metrics_names[1], Pred_conv[1]))

In [ ]:
loaded_model = tensorflow.keras.models.load_model('Model\German_Traffic_DenseNetworkModel_final.h5')
loaded_model.summary()